In [ ]:
# Clear environment
rm(list = ls(all = TRUE))

# Set options
options(stringsAsFactors = FALSE)

# Load packages
library(tidyverse)
library(stargazer)
library(fixest)
library(devtools)
library(bartik.weight)
library(lfe)
library(tidyr)
library(ssaggregate)
library(knitr)
library(kableExtra)

In [ ]:
# Data
q1data <- read.csv('q1data.csv')
q3data <- read.csv('q3data.csv')
shifts_shares <- read.csv('shifts_shares.csv')
sector_shares_wide <- read.csv('sector_shares_wide.csv')
sector_shares_long <- read.csv('sector_shares_long.csv')
industry_totals <- read.csv('industry_totals.csv')

In [ ]:
# 1
reg1 <- felm(formula = delta_gdp ~ delta_B, data = q1data, cluster ~ czone)

stargazer(reg1,
          type = "text",
          dep.var.labels = c("Change in Ln(GDP), 2012-15"),
          covariate.labels = c("Change in B, 2012-15"),
          table.placement = "!htbp",
          add.lines = list(c('Clustering level', 'CZ')),
          omit.stat = c('adj.rsq', 'ser'),
          out = 'HW2q1.tex',
          notes.align = "l")

In [ ]:
# 3

reg2 <- felm(delta_emp_1215 ~ delta_B, data = q3data, cluster ~ czone)
reg3 <- felm(delta_emp_1215 ~ delta_B| state, data = q3data, cluster ~ czone)
reg4 <- felm(delta_emp_1215 ~ delta_B, data = q3data, cluster ~ state)
reg5 <- felm(delta_emp_1215 ~ delta_B| state, data = q3data, cluster ~ state)

f_stats <- c(
  waldtest(reg2, ~ delta_B)[5],
  waldtest(reg3, ~ delta_B)[5],
  waldtest(reg4, ~ delta_B)[5],
  waldtest(reg5, ~ delta_B)[5]
)

stargazer(
  reg2, reg3, reg4, reg5,
  type = 'text',
  dep.var.labels = c('Change in CZ employment, 2012-15'),
  digits = 2,
  covariate.labels = c('Change in B, 2012-15'),
  add.lines = list(
    c('Clustering level', 'CZ', 'CZ', 'State', 'State'),
    c('State FE', 'No', 'Yes', 'No', 'Yes'),
    c('F-Stat on Bartik IV', round(f_stats, 2))
  ),
  no.space = TRUE,
  column.sep.width = "1pt",
  notes.append = FALSE,
  notes.align = "l",
  table.placement = "!htbp",
  omit.stat = c('adj.rsq', 'ser'),
  out = 'HW2q3.tex'
)

In [ ]:
# 4

# Prepare variables in the master tibble
y = "delta_gdp"
x = "delta_emp_1215"

# Prepare variables in the local tibble
Z = setdiff(names(sector_shares_wide), c('X', "czone", 'state'))

# Prepare variables in the global tibble
G = "growth"

# Estimate the weight (alpha) and the IV estimates (beta)
bw = bw(q3data, y, x, NULL, NULL, sector_shares_wide, Z, industry_totals, G)
q4table <- head(bw[order(bw$alpha, decreasing = TRUE), ], 5) %>% select(sector, alpha, beta)
q4table %>%
  kable("latex", booktabs = TRUE, digits = 3) %>%
  kable_styling(full_width = FALSE, font_size = 12)

In [ ]:
# 5

# first need to transform things using ssaggregate, and then do the regression. should obtain the same
# estimate as 3 but different standard errors

industry = ssaggregate(
  data = q3data,
  shares = sector_shares_long,
  vars = ~ delta_gdp + delta_emp_1215 + delta_B ,
  n = "sector",
  s = "share",
  l = 'czone',
  addmissing = TRUE
)

reg6 <- feols(delta_emp_1215 ~ delta_B, data = industry,
              weights = ~ s_n, vcov = "hc1")

etable(reg6,
       title = "Corrected estimates for Table 2, column 1",
       notes = "HC1 robust standard errors",
       tex = TRUE,
       file = "HW2q5.tex")